In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
dataset = pd.read_csv("US30M5.csv", low_memory=False)

In [5]:
dataset

,DateTime,Open,High,Low,Close,TickVolume,DONOTUSE,Pivot,PivotType
0,2019.10.08 01:00,26459.4,26464.4,26448.4,26456.4,261,0,0,NaN
1,2019.10.08 01:05,26456.9,26461.4,26453.9,26460.4,78,0,0,NaN
2,2019.10.08 01:10,26460.9,26471.4,26460.4,26469.9,105,0,0,NaN
3,2019.10.08 01:15,26470.4,26473.9,26468.4,26471.4,82,0,0,NaN
4,2019.10.08 01:20,26470.9,26476.9,26470.4,26476.4,56,0,0,NaN
...,...,...,...,...,...,...,...,...,...
99995,2021.02.19 04:10,31415.0,31417.0,31402.5,31406.0,293,0,0,NaN
99996,2021.02.19 04:15,31406.5,31407.5,31379.0,31379.5,466,0,0,NaN
99997,2021.02.19 04:20,31380.5,31388.0,31373.0,31384.5,501,0,1,Trough
99998,2021.02.19 04:25,31383.5,31398.0,31376.5,31396.0,439,0,0,NaN


In [4]:
# generate pivot point flag
# definition of pivot points is:
# high is higher than previous high and next high, low is higher than previous low and next low -> peak
# low is lower than previous low and next low, high is lower than previous high and next high -> trough

# create pivot point array where 0 means not a pivot point and 1 means is a pivot point
pivot = [0]
pivotType = [np.nan]
for i in range(1, len(dataset)-1):
    print(str((i)/(len(dataset)-2)*100) + "%", end="\r")
    # check whether point is a peak
    if (float(dataset.loc[i]["High"]) > float(dataset.loc[i+1]["High"])) and (float(dataset.loc[i]["High"]) > float(dataset.loc[i-1]["High"])) and (float(dataset.loc[i]["Low"]) > float(dataset.loc[i+1]["Low"])) and (float(dataset.loc[i]["Low"]) > float(dataset.loc[i-1]["Low"])):
        pivot.append(1)
        pivotType.append("Peak")
    # check whether point is a trough
    elif (float(dataset.loc[i]["High"]) < float(dataset.loc[i+1]["High"])) and (float(dataset.loc[i]["High"]) < float(dataset.loc[i-1]["High"])) and (float(dataset.loc[i]["Low"]) < float(dataset.loc[i+1]["Low"])) and (float(dataset.loc[i]["Low"]) < float(dataset.loc[i-1]["Low"])):
        pivot.append(1)
        pivotType.append("Trough")
    # not a pivot point
    else:
        pivot.append(0)
        pivotType.append(np.nan)
# append the final value as not pivot
pivot.append(0)
pivotType.append(np.nan)

# append pivot and pivotType columns
try:
    dataset.insert(len(dataset.columns), "Pivot", pivot)
    dataset.insert(len(dataset.columns), "PivotType", pivotType)
except ValueError:
    print("Pivot and/or PivotType columns already exist in dataset")

In [6]:
# Generate candle type flag
# Flag is Higher when close is higher than open, Lower when close is lower than open, and Doji when close is equal to open
candleType = []
for i in range(len(dataset)):
    print(str(i+1) + "/" + str(len(dataset)), end="\r")
    if float(dataset.loc[i]["Close"]) > float(dataset.loc[i]["Open"]):
        candleType.append("Higher")
    elif float(dataset.loc[i]["Close"]) < float(dataset.loc[i]["Open"]):
        candleType.append("Lower")
    else:
        candleType.append("Doji")
        
# append pivot and pivotType columns
try:
    dataset.insert(len(dataset.columns), "CandleType", candleType)
except ValueError:
    print("CandleType column already exist in dataset")

In [9]:
# generate dataset of horizontal support and resistance lines
# dataset includes start datetime and price level at each horizontal line, as well as number of rejections from each line
# a horizontal trendline is defined as the price level at a pivot point
# a rejection is defined as either the low reaching lower than horizontal price level, and close is higher than horizontal price level
# or high reaching higher than horizontal price level, and close is lower than horizontal price level
# price level of each horizontal line is the high of a pivot point if pivot point type = 'Peak', and low of a pivot point if pivot point type = 'Trough'

# create all possible horizontal support and resistance lines
horizontalLines = dataset.loc[dataset["Pivot"] == 1].reset_index(drop=True)
horizontalLines["PriceLevel"] = np.where(horizontalLines["PivotType"] == "Peak", horizontalLines["High"], horizontalLines["Low"])

# drop columns
horizontalLines.drop(labels=["Open", "High", "Low", "Close", "TickVolume", "DONOTUSE", "Pivot", "PivotType", "CandleType"], axis="columns")

RejectionCount = pd.Series([0 for i in range(len(horizontalLines))])
SupportRejectionCount = pd.Series([0 for i in range(len(horizontalLines))]) # candle has low lower than or equal to horizontal price level, and close higher than horizontal price level
ResistanceRejectionCount = pd.Series([0 for i in range(len(horizontalLines))]) # candle has high higher than or equal to horizontal price level, and close lower than horizontal price level
startIndex = int(dataset.loc[dataset['DateTime'] == str(horizontalLines.iloc[0]['DateTime'])].index[0]) + 1

# calculate rejections for each horizontal line using entire candle chart dataset
for i in range(startIndex, len(dataset)):
    print(str(i) + "/" + str(len(dataset)), end="\r")
    currentTime = str(dataset.loc[i]["DateTime"])
    rangeHigh = float(dataset.loc[i]["High"])
    rangeLow = float(dataset.loc[i]["Close"])
    # indices of horizontal lines where resistance rejection occurs
    indices = horizontalLines.loc[(horizontalLines["PriceLevel"] <= rangeHigh) & (horizontalLines["PriceLevel"] > rangeLow) & (horizontalLines["DateTime"] < currentTime)].index
    # increment rejection count
    RejectionCount.iloc[indices] += 1
    ResistanceRejectionCount.iloc[indices] += 1
    rangeHigh = float(dataset.loc[i]["Close"])
    rangeLow = float(dataset.loc[i]["Low"])
    # indices of horizontal lines where support rejection occurs
    indices = horizontalLines.loc[(horizontalLines["PriceLevel"] >= rangeLow) & (horizontalLines["PriceLevel"] < rangeHigh) & (horizontalLines["DateTime"] < currentTime)].index
    # increment rejection count
    RejectionCount.iloc[indices] += 1
    SupportRejectionCount.iloc[indices] += 1
    
try:
    horizontalLines.insert(len(horizontalLines.columns), "RejectionCount", RejectionCount)
    horizontalLines.insert(len(horizontalLines.columns), "SupportRejectionCount", SupportRejectionCount)
    horizontalLines.insert(len(horizontalLines.columns), "ResistanceRejectionCount", ResistanceRejectionCount)
except ValueError:
    print("RejectionCount and/or SupportRejectionCount and/or ResistanceRejectionCount already exists in table horizontalLines")

In [49]:
startIndex = int(dataset.loc[dataset['DateTime'] == str(horizontalLines.iloc[0]['DateTime'])].index[0]) + 1

In [50]:
startIndex

13

In [10]:
horizontalLines

,DateTime,Open,High,Low,Close,TickVolume,DONOTUSE,Pivot,PivotType,CandleType,PriceLevel,RejectionCount,SupportRejectionCount,ResistanceRejectionCount
0,2019.10.08 02:00,26470.9,26471.9,26463.4,26463.9,103,0,1,Trough,Lower,26463.4,378,206,172
1,2019.10.08 02:15,26476.4,26481.4,26475.9,26479.9,48,0,1,Peak,Higher,26481.4,422,215,207
2,2019.10.08 02:45,26475.4,26484.9,26474.4,26479.9,106,0,1,Peak,Higher,26484.9,437,241,196
3,2019.10.08 03:20,26493.4,26497.4,26487.9,26492.4,335,0,1,Peak,Lower,26497.4,463,253,210
4,2019.10.08 03:25,26492.9,26495.9,26484.4,26491.4,291,0,1,Trough,Lower,26484.4,426,233,193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18974,2021.02.19 02:00,31472.0,31485.0,31468.0,31475.0,498,0,1,Peak,Higher,31485.0,7,3,4
18975,2021.02.19 02:20,31484.0,31493.0,31478.0,31488.0,387,0,1,Peak,Higher,31493.0,2,1,1
18976,2021.02.19 02:50,31479.5,31480.5,31470.0,31476.0,200,0,1,Trough,Lower,31470.0,2,1,1
18977,2021.02.19 03:45,31409.5,31414.0,31399.5,31411.0,546,0,1,Trough,Higher,31399.5,1,0,1


In [61]:
dataset.drop(labels=["CandleType"], axis="columns", inplace=True)

In [59]:
dataset

,DateTime,Open,High,Low,Close,TickVolume,DONOTUSE,Pivot,PivotType,CandleType
0,2019.10.08 01:00,26459.4,26464.4,26448.4,26456.4,261,0,0,NaN,1
1,2019.10.08 01:05,26456.9,26461.4,26453.9,26460.4,78,0,0,NaN,2
2,2019.10.08 01:10,26460.9,26471.4,26460.4,26469.9,105,0,0,NaN,2
3,2019.10.08 01:15,26470.4,26473.9,26468.4,26471.4,82,0,0,NaN,2
4,2019.10.08 01:20,26470.9,26476.9,26470.4,26476.4,56,0,0,NaN,2
5,2019.10.08 01:25,26475.9,26477.4,26465.4,26466.4,47,0,0,NaN,1
6,2019.10.08 01:30,26467.4,26474.9,26467.4,26474.9,66,0,0,NaN,2
7,2019.10.08 01:35,26474.4,26482.4,26474.4,26482.4,85,0,0,NaN,2
8,2019.10.08 01:40,26483.9,26484.4,26475.9,26478.4,62,0,0,NaN,1
9,2019.10.08 01:45,26478.9,26482.9,26478.4,26480.4,55,0,0,NaN,2


In [63]:
horizontalLines.iloc[[1, 2, 3, 4]]

,DateTime,Open,High,Low,Close,TickVolume,DONOTUSE,Pivot,PivotType,PriceLevel
15,2019.10.08 02:15,26476.4,26481.4,26475.9,26479.9,48,0,1,Peak,26481.4
21,2019.10.08 02:45,26475.4,26484.9,26474.4,26479.9,106,0,1,Peak,26484.9
28,2019.10.08 03:20,26493.4,26497.4,26487.9,26492.4,335,0,1,Peak,26497.4
29,2019.10.08 03:25,26492.9,26495.9,26484.4,26491.4,291,0,1,Trough,26484.4


In [67]:
RejectionCount = pd.Series([0 for i in range(len(horizontalLines))])

In [70]:
RejectionCount.iloc[[1, 2, 3, 4]]+=1

In [71]:
RejectionCount

0        0
1        1
2        1
3        1
4        1
5        0
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       0
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
        ..
18949    0
18950    0
18951    0
18952    0
18953    0
18954    0
18955    0
18956    0
18957    0
18958    0
18959    0
18960    0
18961    0
18962    0
18963    0
18964    0
18965    0
18966    0
18967    0
18968    0
18969    0
18970    0
18971    0
18972    0
18973    0
18974    0
18975    0
18976    0
18977    0
18978    0
Length: 18979, dtype: int64